##### Python Reddit Scraper

One-Time Instructions:
1. Sign up for Reddit and login 
2. Goto https://www.reddit.com/prefs/apps
    1. Fill out all fields. Some notes:
        1. Chose 'script' as the app type
        2. For the about/redirect text boxes, choose a website you own or any website
    2. Write down the client_id (2nd line underneath the app title) and client_secret (labeled "secret")
3. Create a text file named ".env" with the following fields:
    - reddit_client_id     = "_FROM STEP 2.2 ABOVE_"
    - reddit_client_secret = "_FROM STEP 2.2 ABOVE_"
    - reddit_user_agent    = "multiplatform:PyScraper:v1.0 (by /u/_YOUR REDDIT USER NAME_)"
    - reddit_username      = "_YOUR REDDIT USER NAME_"
    - reddit_password      = "_YOUR REDDIT PASSWORD_"



In [1]:
# Imports
################################################################################
import os
import praw
import pandas as pd

from   dotenv   import load_dotenv
from   datetime import datetime


In [2]:
# Step 0. Set the global parameters
################################################################################
url_of_interest = 'https://www.reddit.com/r/TikTokCringe/comments/16ir9c7/using_ai_in_schools/'
filename        = 'using-ai-in-schools.csv' # GLOBAL PARAMETER
folder          = "./" # GLOBAL PARAMETER


In [3]:
# Step 1. Login to Reddit (Authentication)
################################################################################

# load all the variables from .env file into environment variables
load_dotenv()  

client_id     = os.getenv("reddit_client_id")
client_secret = os.getenv("reddit_client_secret")
user_agent    = os.getenv("reddit_user_agent")
username      = os.getenv("reddit_username")
password      = os.getenv("reddit_password")

# Note: twoFA and username/password are only needed if posting to reddit
# twoFA = "INSERT TWO-FACTOR AUTHENTICATION" # Unnecessary if only scraping
reddit = praw.Reddit(
    client_id     = client_id,    
    client_secret = client_secret,
    user_agent    = user_agent,   
    #username     = username,               # Unnecessary if only scraping    
    #password     = f'{password}'           # Unnecessary if only scraping
    #password     = f'{password}:{twoFA}'   # If your account has 2FA
)


In [4]:
# Step 1: Scrape the URL of interest
################################################################################

post = reddit.submission(url=url_of_interest)
post.comments.replace_more(limit=None)  # Grab original post
comments = post.comments.list()         # Grab comments

In [5]:
# Step 2 (optional): Check return results
################################################################################

# Print OP
print(f'Original Post:\n')
print(f'Date: {datetime.utcfromtimestamp(post.created_utc).strftime("%Y-%m-%d %H:%M:%S UTC")}\n')
print(f'Title: {post.title}')
print(f'Title: {post.author}')
print(f'Text: {post.selftext}\n')
print(f'Score: {post.score}')
print(f'Replies: {len(comments)}')
print(f'')

# Print all comments of the post
for i, comment in enumerate(comments):
    print(f'\n{i}. {comment.author}\n')
    print(f'Score:{comment.score}')
    print(f'Replies:{len(comment.replies)}')
    print(f'{datetime.utcfromtimestamp(comment.created_utc).strftime("%Y-%m-%d %H:%M:%S UTC")}\n')
    print(f'----------\n{comment.body}\n----------')


Original Post:

Date: 2023-09-14 19:20:13 UTC

Title: Using AI in schools
Title: choganoga
Text: 

Score: 7499
Replies: 316


0. AutoModerator

Score:1
Replies:0
2023-09-14 19:20:30 UTC

----------

**Welcome to r/TikTokCringe!**

 This is a message directed to all newcomers to make you aware that r/TikTokCringe evolved long ago from only cringe-worthy content to TikToks of all kinds! If you’re looking to find only the cringe-worthy TikToks on this subreddit (which are still regularly posted) we recommend sorting by flair which you can do [here](https://www.reddit.com/r/TikTokCringe/comments/galuit/click_here_to_sort_by_flair_a_guide_to_using/) (Currently supported by desktop and reddit mobile). 

 See someone asking how this post is cringe because they didn't read this comment? Show them [this!](https://www.reddit.com/r/TikTokCringe/comments/fyrgzy/for_those_confused_by_the_name_of_this_subreddit/)
    
 Be sure to read the rules of this subreddit before posting or commenting. Thanks!

/tmp/ipykernel_227434/2354340014.py:6: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  print(f'Date: {datetime.utcfromtimestamp(post.created_utc).strftime("%Y-%m-%d %H:%M:%S UTC")}\n')
/tmp/ipykernel_227434/2354340014.py:19: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  print(f'{datetime.utcfromtimestamp(comment.created_utc).strftime("%Y-%m-%d %H:%M:%S UTC")}\n')


In [6]:
# Step 2 (required): Create dataframe of results
################################################################################

thread = []

# Add original post (OP) to the dataframe—customize to add columns of interest
thread.append({
    'reply'   : 0,
    'id'      : post.id,
    'parent_id' : "null",
    'date'    : datetime.utcfromtimestamp(post.created_utc).strftime("%Y-%m-%d %H:%M:%S UTC"),
    'author'  : post.author,
    'title'   : post.title,
    'score'   : post.score,
    'replies' : len(comments),
    'text'    : post.selftext,
})

# Add all comments to the dataframe
for i, comment in enumerate(comments):
    thread.append({
        'reply'     : i+1,
        'id'        : comment.id,
        'parent_id' : comment.parent_id,
        'date'      : datetime.utcfromtimestamp(comment.created_utc).strftime("%Y-%m-%d %H:%M:%S UTC"),
        'author'    : comment.author,
        'title'     : post.title,
        'score'     : comment.score,
        'replies'   : len(comment.replies),
        'text'      : comment.body,
    })

/tmp/ipykernel_227434/512897470.py:11: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  'date'    : datetime.utcfromtimestamp(post.created_utc).strftime("%Y-%m-%d %H:%M:%S UTC"),
/tmp/ipykernel_227434/512897470.py:25: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  'date'      : datetime.utcfromtimestamp(comment.created_utc).strftime("%Y-%m-%d %H:%M:%S UTC"),


In [7]:
# Step 3: Write out dataframe
################################################################################

df       = pd.DataFrame(thread)
fullpath = f'{folder}/{filename}'
df.to_csv(fullpath, index = False)
